# VnV (ver. 3)

- by JPark
- 모델 추가
- 지연 시간 측정을 위한 코드 통합

In [1]:

#------------------------------------------------------
# Config
#------------------------------------------------------

# Test images

zip_images_url = 'http://keticmr.iptime.org:22080/edgeai/images/imagenet-mini-val.zip'
zip_images = 'imagenet-mini-val.zip'
dataset_root = './dataset'
fpath_zip_images = dataset_root + '/' + zip_images
fpath_testimages = dataset_root + '/imagenet-mini-val/'

# Models
urlroot = 'http://keticmr.iptime.org:22080/edgeai/models_jpark/'
modeldir = './checkpoint/'
    
model_names_resnet = [ 
    'resnet18',
    'resnet34',
    'resnet50',
    'resnet101',
    'resnet152',
]
repo_resnet = 'pytorch/vision:v0.10.0'

model_names_mobnet = [
    'mobilenet_v3_small',
    'mobilenet_v3_large',
]
repo_mobnet = 'pytorch/vision:v0.10.0'

'''
model_names_effnet = [ 
    'nvidia_efficientnet_b0',
    'nvidia_efficientnet_b4',
    'nvidia_efficientnet_widese_b0',
    'nvidia_efficientnet_widese_b4',
]
repo_effnet = 'NVIDIA/DeepLearningExamples:torchhub'
'''

from efficientnet_pytorch import EfficientNet
model_names_effnet = [ 
    'efficientnet-b0',
    'efficientnet-b1',
    'efficientnet-b2',
    'efficientnet-b3',
    'efficientnet-b4',
    'efficientnet-b5',
    'efficientnet-b6',
    'efficientnet-b7',
]
repo_effnet = ''



model_names = model_names_resnet + model_names_effnet + model_names_mobnet
#model_names = model_names_effnet


pth_names = [ model_name + '-dict.pth' for model_name in model_names ]

urlmodels = []
for pth_name in pth_names:
    urlmodels.append(urlroot + pth_name)

model_fpaths = []
for pth_name in pth_names:
    model_fpaths.append(modeldir + pth_name)


In [2]:
pth_names

['resnet18-dict.pth',
 'resnet34-dict.pth',
 'resnet50-dict.pth',
 'resnet101-dict.pth',
 'resnet152-dict.pth',
 'efficientnet-b0-dict.pth',
 'efficientnet-b1-dict.pth',
 'efficientnet-b2-dict.pth',
 'efficientnet-b3-dict.pth',
 'efficientnet-b4-dict.pth',
 'efficientnet-b5-dict.pth',
 'efficientnet-b6-dict.pth',
 'efficientnet-b7-dict.pth',
 'mobilenet_v3_small-dict.pth',
 'mobilenet_v3_large-dict.pth']

In [3]:
urlmodels

['http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet18-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet34-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet50-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet101-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet152-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b0-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b1-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b2-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b3-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b4-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b5-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficientnet-b6-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/efficie

In [4]:
model_fpaths

['./checkpoint/resnet18-dict.pth',
 './checkpoint/resnet34-dict.pth',
 './checkpoint/resnet50-dict.pth',
 './checkpoint/resnet101-dict.pth',
 './checkpoint/resnet152-dict.pth',
 './checkpoint/efficientnet-b0-dict.pth',
 './checkpoint/efficientnet-b1-dict.pth',
 './checkpoint/efficientnet-b2-dict.pth',
 './checkpoint/efficientnet-b3-dict.pth',
 './checkpoint/efficientnet-b4-dict.pth',
 './checkpoint/efficientnet-b5-dict.pth',
 './checkpoint/efficientnet-b6-dict.pth',
 './checkpoint/efficientnet-b7-dict.pth',
 './checkpoint/mobilenet_v3_small-dict.pth',
 './checkpoint/mobilenet_v3_large-dict.pth']

In [5]:
fpath_zip_images

'./dataset/imagenet-mini-val.zip'

In [6]:
model_names

['resnet18',
 'resnet34',
 'resnet50',
 'resnet101',
 'resnet152',
 'efficientnet-b0',
 'efficientnet-b1',
 'efficientnet-b2',
 'efficientnet-b3',
 'efficientnet-b4',
 'efficientnet-b5',
 'efficientnet-b6',
 'efficientnet-b7',
 'mobilenet_v3_small',
 'mobilenet_v3_large']

## 00. 사전작업

- 사전으로 실험을 위한 디렉토리를 생성하고, 추론 영상을 다운로드 받습니다. (변인통제 요소)

In [7]:

#------------------------------------------------------
# Download data
#------------------------------------------------------
import os
import urllib

# make download directory
def makedir(path): 
    isdir = os.path.isdir(path)
    
    try: 
        os.makedirs(path)
    except OSError: 
        if not isdir: 
            raise
    return os.path.abspath(path), isdir

# Download images
d, isdir = makedir(dataset_root) # 저장 공간 생성

if isdir:
    url, fname = (zip_images_url, fpath_zip_images)
    isfile_exist = os.path.exists(os.path.join(os.getcwd(), fname))
    
    print('downloading...')
    if not isfile_exist:
        try: 
            urllib.URLopener().retrieve(url, fname)
        except: 
            urllib.request.urlretrieve(url, fname)
        print('[+] download completed.')
        # Unzip
        cmd = 'unzip ' + fpath_zip_images + ' -d ' + dataset_root
        print(cmd)
        os.system(cmd)
    else:
        print('[+] download skipped ')


from glob import iglob

'''
# read test files
testfiles = []
for fname in sorted( iglob(fpath_testimages + '**/*.JPEG', recursive=True) ):
    testfiles.append(fname)
'''

idx_gt = []
idx = 0
testfiles = []
for d in sorted( iglob(fpath_testimages + 'n*', recursive=False) ):
    for fname in sorted( iglob(d + '/*.JPEG', recursive=True) ):
        testfiles.append(fname)
        idx_gt.append( idx )
    idx += 1


# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]


downloading...
[+] download completed.
unzip ./dataset/imagenet-mini-val.zip -d ./dataset
Archive:  ./dataset/imagenet-mini-val.zip
   creating: ./dataset/imagenet-mini-val/
  inflating: ./dataset/__MACOSX/._imagenet-mini-val  
   creating: ./dataset/imagenet-mini-val/n04542943/
  inflating: ./dataset/__MACOSX/imagenet-mini-val/._n04542943  
   creating: ./dataset/imagenet-mini-val/n02795169/
  inflating: ./dataset/__MACOSX/imagenet-mini-val/._n02795169  
   creating: ./dataset/imagenet-mini-val/n02422699/
  inflating: ./dataset/__MACOSX/imagenet-mini-val/._n02422699  
   creating: ./dataset/imagenet-mini-val/n03498962/
  inflating: ./dataset/__MACOSX/imagenet-mini-val/._n03498962  
   creating: ./dataset/imagenet-mini-val/n01740131/
  inflating: ./dataset/__MACOSX/imagenet-mini-val/._n01740131  
   creating: ./dataset/imagenet-mini-val/n02769748/
  inflating: ./dataset/__MACOSX/imagenet-mini-val/._n02769748  
   creating: ./dataset/imagenet-mini-val/n02319095/
  inflating: ./dataset/_

## 01. 실행

In [8]:
len(idx_gt)

3923

## GPU 동작모드

In [9]:
i=4
print (f'{i:04d}')

0004


In [10]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time
from tqdm import tqdm
import numpy as np
import torch

'''
# Define transforms for the evaluation phase
preprocess = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])
'''

preproc = ['method1', 'method2']
preproc_method = 'method1'

#devices = ['cuda', 'cpu']
devices = ['cuda']

models = []

testset = testfiles[:]
n = len(testset)

# 디바이스별 반복
for device in devices: 
    print('-'*50)
    print('device = ', device, flush=True)
    print('-'*50)
        
    # 모델별 반복
    for model_idx, model_name in enumerate(model_names):
        start = time.time() # strt timer        
        print(f'model = {model_names[model_idx]}', flush=True)
        
        # 모델 템플릿 다운로드 (from torch.hub)
        if model_name in model_names_resnet:
            model = torch.hub.load(repo_resnet, model_name, pretrained=False)
        elif model_name in model_names_mobnet:
            model = torch.hub.load(repo_mobnet, model_name, pretrained=False)
        elif model_name in model_names_effnet:
            #model = torch.hub.load(repo_effnet, model_name, pretrained=False)
            model = EfficientNet.from_pretrained(model_name)
        else:
            pass
    
        # 모델 가중치 다운로드 (from AI 모델 리포지토리)
        if False:    
            # 방법 1
            pass
        
            # torch.hub.download_url_to_file(urlroot+pthnames[model_idx], modeldir+pthnames[model_idx])
            # model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))
            # model.eval()
        else:
            # 방법 2
            checkpoint = urlmodels[model_idx]
            model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))
            model.eval().to(device) # change model to evauation mode (e.g. disable Dropout, BatchNorm)
            models.append(model)
 
        # 시험용 입력 영상
        top1_cnt = 0
        top5_cnt = 0
        top1_catids = []
              
        # 시험용 입력 영상 전처리 (크기 및 컬러채널)
        if preproc_method == preproc[0]:
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

        # 시험영상별 반복
        imgidx = 0
        for fpath in tqdm( testset ):
        #for fpath in testset:
            #print( fpath )
            input_image = Image.open(fpath)

            try:
                input_tensor = preprocess(input_image)
            except:
                # gray scale to color
                input_image = Image.open(fpath).convert("RGB")
                input_tensor = preprocess(input_image)

            input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

            # 디바이스 설정
            if device == 'cuda':
                if torch.cuda.is_available():
                    input_batch = input_batch.to('cuda')
                    model.to('cuda')
            else:
                input_batch = input_batch.to('cpu')
                model.to('cpu')
              
            with torch.no_grad():
                output = model(input_batch)

            # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
            #print(output[0])

            # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            #print(probabilities)

            # Show top categories per image
            top5_prob, top5_catid = torch.topk(probabilities, 5)
            for i in range(top5_prob.size(0)):
                #print(top5_catid[i])
                #print(imgidx, ' ', categories[top5_catid[i]], top5_prob[i].item())

                if( top5_catid[i] == idx_gt[imgidx] ):
                    top5_cnt += 1
                    
            # Show top categories per image
            top1_prob, top1_catid = torch.topk(probabilities, 1)
            top1_catids.append(top1_catid[0])
            if( top1_catid[0] == idx_gt[imgidx] ):
                top1_cnt += 1
 
            imgidx += 1

        end = time.time() # end timer
        print('n = ', n)
        print('top1_cnt = ', top1_cnt)
        print('top1_acc = ', top1_cnt/n)
        print('top5_cnt = ', top5_cnt)
        print('top5_acc = ', top5_cnt/n)
        print('time = ', end - start)
        print('')
        
        
        print('-' * 70)
        print('GT')
        print('-' * 70)
        for idx, cid in enumerate(idx_gt):
            print(f'{idx:04d} = {cid}')
            
        print('-' * 70)
        print('Top1 category IDs')
        print('-' * 70)
        for idx, cid in enumerate(top1_catids):
            print(f'{idx:04d} = {int(cid)}')

--------------------------------------------------
device =  cuda
--------------------------------------------------
model = resnet18


Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jpark/.virtualenvs/dl4cv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jpark/.virtualenvs/dl4cv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet18-dict.pth" to /home/jpark/.cache/torch/hub/checkpoints/resnet18-dict.pth
100%|██████████| 3923/3923 [00:20<00:00, 193.79it/s]

n =  3923
top1_cnt =  2500
top1_acc =  0.637267397399949
top5_cnt =  3358
top5_acc =  0.8559775681876115
time =  24.556164503097534

----------------------------------------------------------------------
GT
----------------------------------------------------------------------
0000 = 0
0001 = 0
0002 = 0
0003 = 1
0004 = 1
0005 = 1
0006 = 2
0007 = 2
0008 = 2
0009 = 2
0010 = 2
0011 = 3
0012 = 3
0013 = 3
0014 = 4
0015 = 4
0016 = 4
0017 = 4
0018 = 4
0019 = 5
0020 = 5
0021 = 5
0022 = 6
0023 = 6
0024 = 6
0025 = 7
0026 = 7
0027 = 7
0028 = 8
0029 = 8
0030 = 8
0031 = 9
0032 = 9
0033 = 9
0034 = 10
0035 = 10
0036 = 10
0037 = 10
0038 = 11
0039 = 11
0040 = 11
0041 = 11
0042 = 12
0043 = 12
0044 = 12
0045 = 13
0046 = 13
0047 = 13
0048 = 13
0049 = 14
0050 = 14
0051 = 14
0052 = 14
0053 = 15
0054 = 15
0055 = 15
0056 = 16
0057 = 16
0058 = 16
0059 = 17
0060 = 17
0061 = 17
0062 = 18
0063 = 18
0064 = 19
0065 = 19
0066 = 19
0067 = 19
0068 = 20
0069 = 20
0070 = 20
0071 = 21
0072 = 21
0073 = 21
0074 = 21
0075 =


Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet34-dict.pth" to /home/jpark/.cache/torch/hub/checkpoints/resnet34-dict.pth
100%|██████████| 3923/3923 [00:21<00:00, 185.98it/s]

n =  3923
top1_cnt =  2690
top1_acc =  0.6856997196023451
top5_cnt =  3476
top5_acc =  0.8860565893448891
time =  28.724655151367188

----------------------------------------------------------------------
GT
----------------------------------------------------------------------
0000 = 0
0001 = 0
0002 = 0
0003 = 1
0004 = 1
0005 = 1
0006 = 2
0007 = 2
0008 = 2
0009 = 2
0010 = 2
0011 = 3
0012 = 3
0013 = 3
0014 = 4
0015 = 4
0016 = 4
0017 = 4
0018 = 4
0019 = 5
0020 = 5
0021 = 5
0022 = 6
0023 = 6
0024 = 6
0025 = 7
0026 = 7
0027 = 7
0028 = 8
0029 = 8
0030 = 8
0031 = 9
0032 = 9
0033 = 9
0034 = 10
0035 = 10
0036 = 10
0037 = 10
0038 = 11
0039 = 11
0040 = 11
0041 = 11
0042 = 12
0043 = 12
0044 = 12
0045 = 13
0046 = 13
0047 = 13
0048 = 13
0049 = 14
0050 = 14
0051 = 14
0052 = 14
0053 = 15
0054 = 15
0055 = 15
0056 = 16
0057 = 16
0058 = 16
0059 = 17
0060 = 17
0061 = 17
0062 = 18
0063 = 18
0064 = 19
0065 = 19
0066 = 19
0067 = 19
0068 = 20
0069 = 20
0070 = 20
0071 = 21
0072 = 21
0073 = 21
0074 = 21
0075 

...

n =  3923
top1_cnt =  2880
top1_acc =  0.7341320418047412
top5_cnt =  3588
top5_acc =  0.9146061687484068
time =  82.5811812877655

----------------------------------------------------------------------
GT
----------------------------------------------------------------------
0000 = 0
0001 = 0
0002 = 0
0003 = 1
0004 = 1
0005 = 1
0006 = 2
0007 = 2
0008 = 2
0009 = 2
0010 = 2
0011 = 3
0012 = 3
0013 = 3
0014 = 4
0015 = 4
0016 = 4
0017 = 4
0018 = 4
0019 = 5
0020 = 5
0021 = 5
0022 = 6
0023 = 6
0024 = 6
0025 = 7
0026 = 7
0027 = 7
0028 = 8
0029 = 8
0030 = 8
0031 = 9
0032 = 9
0033 = 9
0034 = 10
0035 = 10
0036 = 10
0037 = 10
0038 = 11
0039 = 11
0040 = 11
0041 = 11
0042 = 12
0043 = 12
0044 = 12
0045 = 13
0046 = 13
0047 = 13
0048 = 13
0049 = 14
0050 = 14
0051 = 14
0052 = 14
0053 = 15
0054 = 15
0055 = 15
0056 = 16
0057 = 16
0058 = 16
0059 = 17
0060 = 17
0061 = 17
0062 = 18
0063 = 18
0064 = 19
0065 = 19
0066 = 19
0067 = 19
0068 = 20
0069 = 20
0070 = 20
0071 = 21
0072 = 21
0073 = 21
0074 = 21
0075 = 


Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "http://keticmr.iptime.org:22080/edgeai/models_jpark/mobilenet_v3_small-dict.pth" to /home/jpark/.cache/torch/hub/checkpoints/mobilenet_v3_small-dict.pth
100%|██████████| 3923/3923 [00:22<00:00, 178.02it/s]

n =  3923
top1_cnt =  2527
top1_acc =  0.6441498852918685
top5_cnt =  3353
top5_acc =  0.8547030333928116
time =  23.021501779556274

----------------------------------------------------------------------
GT
----------------------------------------------------------------------
0000 = 0
0001 = 0
0002 = 0
0003 = 1
0004 = 1
0005 = 1
0006 = 2
0007 = 2
0008 = 2
0009 = 2
0010 = 2
0011 = 3
0012 = 3
0013 = 3
0014 = 4
0015 = 4
0016 = 4
0017 = 4
0018 = 4
0019 = 5
0020 = 5
0021 = 5
0022 = 6
0023 = 6
0024 = 6
0025 = 7
0026 = 7
0027 = 7
0028 = 8
0029 = 8
0030 = 8
0031 = 9
0032 = 9
0033 = 9
0034 = 10
0035 = 10
0036 = 10
0037 = 10
0038 = 11
0039 = 11
0040 = 11
0041 = 11
0042 = 12
0043 = 12
0044 = 12
0045 = 13
0046 = 13
0047 = 13
0048 = 13
0049 = 14
0050 = 14
0051 = 14
0052 = 14
0053 = 15
0054 = 15
0055 = 15
0056 = 16
0057 = 16
0058 = 16
0059 = 17
0060 = 17
0061 = 17
0062 = 18
0063 = 18
0064 = 19
0065 = 19
0066 = 19
0067 = 19
0068 = 20
0069 = 20
0070 = 20
0071 = 21
0072 = 21
0073 = 21
0074 = 21
0075 


Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "http://keticmr.iptime.org:22080/edgeai/models_jpark/mobilenet_v3_large-dict.pth" to /home/jpark/.cache/torch/hub/checkpoints/mobilenet_v3_large-dict.pth
100%|██████████| 3923/3923 [00:21<00:00, 184.82it/s]


n =  3923
top1_cnt =  2811
top1_acc =  0.7165434616365026
top5_cnt =  3556
top5_acc =  0.9064491460616875
time =  23.206194162368774

----------------------------------------------------------------------
GT
----------------------------------------------------------------------
0000 = 0
0001 = 0
0002 = 0
0003 = 1
0004 = 1
0005 = 1
0006 = 2
0007 = 2
0008 = 2
0009 = 2
0010 = 2
0011 = 3
0012 = 3
0013 = 3
0014 = 4
0015 = 4
0016 = 4
0017 = 4
0018 = 4
0019 = 5
0020 = 5
0021 = 5
0022 = 6
0023 = 6
0024 = 6
0025 = 7
0026 = 7
0027 = 7
0028 = 8
0029 = 8
0030 = 8
0031 = 9
0032 = 9
0033 = 9
0034 = 10
0035 = 10
0036 = 10
0037 = 10
0038 = 11
0039 = 11
0040 = 11
0041 = 11
0042 = 12
0043 = 12
0044 = 12
0045 = 13
0046 = 13
0047 = 13
0048 = 13
0049 = 14
0050 = 14
0051 = 14
0052 = 14
0053 = 15
0054 = 15
0055 = 15
0056 = 16
0057 = 16
0058 = 16
0059 = 17
0060 = 17
0061 = 17
0062 = 18
0063 = 18
0064 = 19
0065 = 19
0066 = 19
0067 = 19
0068 = 20
0069 = 20
0070 = 20
0071 = 21
0072 = 21
0073 = 21
0074 = 21
0075 

In [12]:
int(top1_catids[111])

71

## CPU 동작모드

In [13]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time
from tqdm import tqdm
import numpy as np
import torch

'''
# Define transforms for the evaluation phase
preprocess = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])
'''

preproc = ['method1', 'method2']
preproc_method = 'method1'

devices = ['cuda', 'cpu']
devices = ['cpu']

models = []


testset = testfiles[:]
n = len(testset)

# 디바이스별 반복
for device in devices: 
    print('-'*50)
    print('device = ', device, flush=True)
    print('-'*50)
        
    # 모델별 반복
    for model_idx, model_name in enumerate(model_names):
        start = time.time() # strt timer        
        print(f'model = {model_names[model_idx]}', flush=True)
        
        # 모델 템플릿 다운로드 (from torch.hub)
        if model_name in model_names_resnet:
            model = torch.hub.load(repo_resnet, model_name, pretrained=False)
        elif model_name in model_names_mobnet:
            model = torch.hub.load(repo_mobnet, model_name, pretrained=False)
        elif model_name in model_names_effnet:
            model = torch.hub.load(repo_effnet, model_name, pretrained=False)
        else:
            pass
    
        # 모델 가중치 다운로드 (from AI 모델 리포지토리)
        if False:    
            # 방법 1
            pass
        
            # torch.hub.download_url_to_file(urlroot+pthnames[model_idx], modeldir+pthnames[model_idx])
            # model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))
            # model.eval()
        else:
            # 방법 2
            checkpoint = urlmodels[model_idx]
            model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))
            model.eval().to(device) # change model to evauation mode (e.g. disable Dropout, BatchNorm)
            models.append(model)
 
        # 시험용 입력 영상
        top1_cnt = 0
        top5_cnt = 0
              
        # 시험용 입력 영상 전처리 (크기 및 컬러채널)
        if preproc_method == preproc[0]:
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

        # 시험영상별 반복
        imgidx = 0
        for fpath in tqdm( testset ):
        #for fpath in testset:
            #print( fpath )
            input_image = Image.open(fpath)

            try:
                input_tensor = preprocess(input_image)
            except:
                # gray scale to color
                input_image = Image.open(fpath).convert("RGB")
                input_tensor = preprocess(input_image)

            input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

            # 디바이스 설정
            if device == 'cuda':
                if torch.cuda.is_available():
                    input_batch = input_batch.to('cuda')
                    model.to('cuda')
            else:
                input_batch = input_batch.to('cpu')
                model.to('cpu')
              
            with torch.no_grad():
                output = model(input_batch)

            # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
            #print(output[0])

            # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            #print(probabilities)

            # Show top categories per image
            top5_prob, top5_catid = torch.topk(probabilities, 5)
            for i in range(top5_prob.size(0)):
                #print(top5_catid[i])
                #print(imgidx, ' ', categories[top5_catid[i]], top5_prob[i].item())

                if( top5_catid[i] == idx_gt[imgidx] ):
                    top1_cnt += 1

            imgidx += 1

        end = time.time() # end timer
        print('n = ', n)
        print('top1_cnt = ', top1_cnt)
        print('top1_cnt/n = ', top1_cnt/n)
        print('time = ', end - start)
        print('')

--------------------------------------------------
device =  cpu
--------------------------------------------------
model = resnet18


Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 3923/3923 [00:53<00:00, 73.19it/s]

n =  3923
top1_cnt =  3358
top1_cnt/n =  0.8559775681876115
time =  53.6722137928009

model = resnet34



Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 3923/3923 [01:24<00:00, 46.64it/s]

n =  3923
top1_cnt =  3476
top1_cnt/n =  0.8860565893448891
time =  84.25164771080017

model = resnet50


...


Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 3923/3923 [03:04<00:00, 21.25it/s]

n =  3923
top1_cnt =  3572
top1_cnt/n =  0.9105276574050472
time =  184.98087763786316

model = resnet152



Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 3923/3923 [04:28<00:00, 14.63it/s]

n =  3923
top1_cnt =  3584
top1_cnt/n =  0.9135865409125669
time =  268.62847089767456

model = efficientnet-b0


ValueError: not enough values to unpack (expected 2, got 1)